## reference
引用的抽象类，定义了所有引用对象的基本操作。因为引用对象的实现和垃圾收集器紧密相关，所以该类可能没有直接的子类实现。
Abstract base class for reference objects. This class defines the operations common to all reference objects. Because reference objects are implemented in close cooperation with the garbage collector, this class may not be subclassed directly.

方法定义
* get：获取引用对象
* clear：清除引用对象
* isEnqueued：是否入队。若该对象在创建时
* enqueue：入队，将引用加入其所注册的队列中。此方法只被java代码调用，GC在垃圾收集时入队，会直接操作，不调用此方法。

## weak reference
弱引用对象：不会阻止其引用的对象被标记为finalizable、finalized状态，并且最终被GC回收。弱引用对象一般用于标准映射的实现。

finalizer方法：在GC回收对象前执行此方法，一般此方法用于手工释放其它系统资源。 
finalizable对象：jvm的GC最终会执行其finalizer方法。 
finalized状态：对象的finalizer方法已被自动执行。

Weak reference objects, which do not prevent their referents from being made finalizable, finalized, and then reclaimed. Weak references are most often used to implement canonicalizing mappings.

假定某一时间点，GC判定一个对象是弱引用可达的，GC会在同时自动清除所有指向该对象的弱引用对象，并且清除所有该弱引用对象*强引用*和*软引用*可达的弱引用对象。同时，GC会声明之前弱引用指向的对象为finalizable状态。此时或之后，GC会将这些最新清除的弱引用对象入队ReferenceQueue（待回收对象队列）。
Suppose that the garbage collector determines at a certain point in time that an object is weakly reachable. At that time it will atomically clear all weak references to that object and all weak references to any other weakly-reachable objects from which that object is reachable through a chain of strong and soft references. At the same time it will declare all of the formerly weakly-reachable objects to be finalizable. At the same time or at some later time it will enqueue those newly-cleared weak references that are registered with reference queues.




## soft reference
软引用：由垃圾收集器根据内存使用需求，来决定是否被回收。软引用对象经常运用于内存敏感的缓存系统中。
Soft reference objects, which are cleared at the discretion of the garbage collector in response to memory demand. Soft references are most often used to implement memory-sensitive caches.

假设某一时刻，垃圾收集器将某个对象判定为软引用可达的，那么他有可能会自动清理指向该对象的软引用对象，同时，清理指向所有该对象*强引用*可达的其他对象的弱引用。垃圾处理器会在这一时刻，或者之后某一时刻，将这些最新清除的，入队ReferenceQueue（待回收对象队列）。
Suppose that the garbage collector determines at a certain point in time that an object is softly reachable. At that time it may choose to clear atomically all soft references to that object and all soft references to any other softly-reachable objects from which that object is reachable through a chain of strong references.   At the same time or at some later time it will enqueue those newly-cleared soft references that are registered with reference queues.
所有软引用指向的软引用可达对象，在java虚拟机抛出OOM异常之前，一定会被清除。
否则，并没有规定，关于何时应该回收一个软引用可达对象，或者一系列软引用可达对象回收时间顺序。
但是官方鼓励虚拟机实现时，偏向最后收集最新创建，和最近使用的软引用对象。
All soft references to softly-reachable objects are guaranteed to have been cleared before the virtual machine throws an OutOfMemoryError. Otherwise no constraints are placed upon the time at which a soft reference will be cleared or the order in which a set of such references to different objects will be cleared. Virtual machine implementations are, however, encouraged to bias against clearing recently-created or recently-used soft references.
该类主要使用在简单的缓存中，该类或衍生的子类也可能被用于大数据复杂缓存系统中。只要同时存在一个强引用指向软引用指向的对象，那么该对象就不会被回收。凭借此种属性，复杂的缓存系统可以通过设置对缓存组对的强引用，阻止其最新使用的缓存对被丢弃，并且对于其余的缓存组对，垃圾收集器可以自由判断何时回收该对象。
Direct instances of this class may be used to implement simple caches; this class or derived subclasses may also be used in larger data structures to implement more sophisticated caches. As long as the referent of a soft reference is strongly reachable, that is, is actually in use, the soft reference will not be cleared. Thus a sophisticated cache can, for example, prevent its most recently used entries from being discarded by keeping strong referents to those entries, leaving the remaining entries to be discarded at the discretion of the garbage collector.




## FinalReference


## PhantomReference

虚引用：指向的对象在被GC认定为reclaimed之后，即进入待回收队列。虚引用常应用于调度预先检验行为，他是一种比java的finalization机制更灵活的方式。

Phantom reference objects, which are enqueued after the collector determines that their referents may otherwise be reclaimed. Phantom references are most often used for scheduling pre-mortem cleanup actions in a more flexible way than is possible with the Java finalization mechanism.
如果GC在某一时刻判定一个虚引用指向的对象是虚引用可达的，那么在那个时刻或之后，GC将把该对象加入待回收队列。
If the garbage collector determines at a certain point in time that the referent of a phantom reference is phantom reachable, then at that time or at some later time it will enqueue the reference.
为了保证可被回收的对象保持可被回收状态，虚引用对象本身的引用不会被回收，get方法会一直返回null。
In order to ensure that a reclaimable object remains so, the referent of a phantom reference may not be retrieved: The get method of a phantom reference always returns null.
和弱引用，软引用不同，虚引用对象在入队时不会被GC自动回收。一个虚引用可达对象在被清除，或者变得不可达之前，会保持其未被回收的状态。
Unlike soft and weak references, phantom references are not automatically cleared by the garbage collector as they are enqueued. An object that is reachable via phantom references will remain so until all such references are cleared or themselves become unreachable.

## ReferenceQueue
在检测到适当的可达性更改后，垃圾回收器会将注册引用对象附加到引用队列。（即队列里是待回收对象）
 Reference queues, to which registered reference objects are appended by the garbage collector after the appropriate reachability changes are detected.



|    区分\引用类型  |特点 |何时回收引用对象本身    |何时入队引用指向对象     |引用指向对象关联可达对象回收     |虚引用     |          
| -------------      |:-----------:         |:-----------:|:-----------:|:-----------:|
|  强引用          | -Xmx3550m    |虚拟机最大堆内存大小 |
|  软引用         |由垃圾收集器根据内存使用需求，来决定是否被回收  | 某一时刻，GC判断某对象是软引用可达的，可能根据   |由垃圾收集器根据内存使用需求，来决定是否被回收 |
|  弱引用  
|  虚引用   